In [ ]:
import sys
import os
import numpy as np
from pathlib import Path
os.chdir(Path(os.getcwd()).resolve().parents[1])
import setup
from methods import grid_selection_amanda_dynamic
from sklearn.model_selection import ParameterGrid



def writeResults(datasetID, bestScore, bestParams, clfName, distanceMetric, beta=None):
    if distanceMetric == 'BBD' and isinstance(beta,list):
        path = "results/batch/dynamic/gridsearch_amanda_dynamic_{}_EVL-{}-ALLBETAS.txt".format(distanceMetric,clfName)
    elif distanceMetric == 'BBD':
        path = "results/batch/dynamic/gridsearch_amanda_dynamic_{}{:.3f}_EVL-{}.txt".format(distanceMetric, beta,clfName)
    else:
        path = "results/batch/dynamic/gridsearch_amanda_dynamic_{}_EVL-{}.txt".format(distanceMetric,clfName)
        
    file = open(path,"a") 
    string = "{}: {} using {} \n".format(datasetID, bestScore, bestParams)
    file.write(string)
    file.close() 


def main():
    is_windows = sys.platform.startswith('win')
    sep = '\\'
    
    if is_windows == False:
        sep = '/'

    path = os.getcwd()+sep+'data'+sep
    
    #loading datasets
    datasets = [setup.loadCDT, setup.loadCHT, setup.load2CDT, setup.load2CHT, setup.load4CR, setup.load4CRE_V1, 
                setup.load4CRE_V2, setup.load5CVT, setup.loadCSurr, setup.load4CE1CF, setup.loadUG_2C_2D, setup.loadMG_2C_2D, 
                setup.loadFG_2C_2D, setup.loadUG_2C_3D, setup.loadUG_2C_5D, setup.loadGEARS_2C_2D, setup.loadCheckerBoard, 
                setup.loadElecData]
    
    #datasets = [setup.loadNOAADataset]
    #batches = 24
    
    #datasets = [setup.loadKeystroke]
    #batches = 4
    
    #arrClfName = ['SGD', 'NB', 'RF', 'LP', 'KNN']
    arrClfName = ['LP']
    distanceMetric = 'BBD'
    #beta = [-50,-10, -5, -2, -1, -0.5, -0.1, -0.01, -0.001, 1.001, 1.01, 1.1, 1.5, 2, 3, 5, 10, 50]
    beta = 2
    allBetas=True
    for clfName in arrClfName:
        if distanceMetric == 'BBD' and allBetas:
            print("**************** BEGIN of {}-{} results ****************".format(clfName, distanceMetric))
        elif distanceMetric == 'BBD':
            print("**************** BEGIN of {}-{}{:.3f} results ****************".format(clfName, distanceMetric, beta))            
        else:
            print("**************** BEGIN of {}-{} results ****************".format(clfName, distanceMetric))
        batches=47
        poolSize = None
        isBatchMode = True
        #testing grid search
        for i in range(len(datasets)):
            finalScore = 0
            best_grid={}
            dataValues, dataLabels, description = datasets[i](path, sep)

            #Train-test split
            availableQty = int(0.5*len(dataLabels))
            availableLabels = dataLabels[:availableQty] 
            availableData = dataValues[:availableQty]

            # 70/30 train/test data
            initialLabeledData = int(0.1*len(availableLabels))
            sizeOfBatch = int((len(availableLabels)-initialLabeledData)/batches)

            print("{}: {} batches of {} instances".format(description, batches, sizeOfBatch))

            tuned_params = [{"sizeOfBatch":[sizeOfBatch], 
                             "batches":[batches], "poolSize":[poolSize], "isBatchMode":[isBatchMode], 
                             "initialLabeledData":[initialLabeledData], "clfName":[clfName],
                            "distanceMetric":[distanceMetric], "beta":[beta]}]
            if clfName == 'LP' or clfName == 'KNN':
                tuned_params[0].update({"K":[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]})

            for g in ParameterGrid(tuned_params):
                averageAccuracy=0
                gs = grid_selection_amanda_dynamic.run(**g)

                try:
                    gs.fit(availableData, availableLabels)
                    averageAccuracy = np.mean(gs.predict())
                    print(averageAccuracy, g)
                    if finalScore < averageAccuracy:
                        finalScore = averageAccuracy
                        best_grid = g
                except Exception:
                    print("An error occured in ", description, g)
                    #raise Exception

            print(finalScore)
            print(best_grid)
            print("=======================================================================================================")

            writeResults(description, finalScore, best_grid, clfName, distanceMetric, beta)
        
        if distanceMetric == 'BBD' and allBetas:
            print("**************** BEGIN of {}-{} results ****************".format(clfName, distanceMetric))
        elif distanceMetric == 'BBD':
            print("******** END of {}-{}{:.3f} results ********".format(clfName, distanceMetric, beta))
        else:
            print("******** END of {}-{} results ********".format(clfName, distanceMetric))
    
if __name__ == "__main__":
    main()


**************** BEGIN of LP-BBD results ****************
One Class Diagonal Translation. 2 Dimensional data.: 47 batches of 153 instances
99.77787234042556 {'K': 2, 'batches': 47, 'beta': 2, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 153}
99.77787234042556 {'K': 3, 'batches': 47, 'beta': 2, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 153}
99.76382978723406 {'K': 4, 'batches': 47, 'beta': 2, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 153}
99.75000000000001 {'K': 5, 'batches': 47, 'beta': 2, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 153}
99.75000000000001 {'K': 6, 'batches': 47, 'beta': 2, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': 

53.580212765957455 {'K': 8, 'batches': 47, 'beta': 2, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 153}
53.48297872340426 {'K': 9, 'batches': 47, 'beta': 2, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 153}
50.312553191489364 {'K': 10, 'batches': 47, 'beta': 2, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 153}
52.59340425531916 {'K': 11, 'batches': 47, 'beta': 2, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 153}
54.220212765957456 {'K': 12, 'batches': 47, 'beta': 2, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 153}
57.029999999999994
{'K': 4, 'batches': 47, 'beta': 2, 'clfName': 'LP', 'distanceMetric': 